In [ ]:
from gurobipy import *
import random
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from matplotlib import cm

The dataset used for this case study are confidential, instead we share a template of the csv file (see lji_template.csv)

In [ ]:
dat = pd.read_csv('*.csv', sep=';')

Define variables for the problem using the data - separate coefficients for traffickers and victims 

In [ ]:
V = dat.shape[0]
Q = np.log2(dat.iloc[:,12]*dat.iloc[:,10])
Cq = np.ones(V)
num_samples = 1000
# define proportion of trafficking as a random variable
s_mean = dat.iloc[:,3]/dat.iloc[:,2]
s_mean[10]=.31
s_mean[s_mean==0]=.05
mar = .1
s=np.minimum(np.maximum(np.random.triangular(left=s_mean-mar, mode=s_mean, right=s_mean+mar, size=(num_samples, len(s_mean))),0),1)

frac_v = 0.8 # fraction of victims, rest are traffickers

S1_v = [Q*(1-s[j])*frac_v for j in range(num_samples)]
S2_v = [Q*s[j]*frac_v for j in range(num_samples)]

S1_t = [Q*(1-s[j])*(1-frac_v) for j in range(num_samples)]
S2_t = [Q*s[j]*(1-frac_v) for j in range(num_samples)]

num_devices = V
ytpr_v = (dat.iloc[:,5:10] * [.1,.3,.5,.7,.9]).sum(axis=1)/dat.iloc[:,2]
ytpr_t = np.minimum((ytpr_v + 0.05), 1)
ytnr = 0.5 * np.ones(V)

In [ ]:
coeff2_v = [[a*b for a,b in zip(S2_v[j],ytpr_v)] for j in range(num_samples)]
coeff3_v = [[a*b for a,b in zip(S1_v[j],ytnr)] for j in range(num_samples)]

coeff2_t = [[a*b for a,b in zip(S2_t[j],ytpr_t)] for j in range(num_samples)]
coeff3_t = [[a*b for a,b in zip(S1_t[j],ytnr)] for j in range(num_samples)]

coq_v = [S1_v[j]*(1-ytnr)+S2_v[j]*(1-ytpr_v) for j in range(num_samples)]
coq_t = [S1_t[j]*(1-ytnr)+S2_t[j]*(1-ytpr_t) for j in range(num_samples)]

DTRAP-VT for LJI

In [ ]:
alloc_lji = dat.iloc[:,11]
B = sum(dat.iloc[:,11])
K = 60
L = 6*K
m = Model()
m.ModelSense = 1  # minimize
# Add variables
x = m.addVars(V, vtype='I', name='x', lb=[1]*V, ub=[10]*V)
error = m.addVars(num_samples, obj=1.0/num_samples, name='error')
# Set constraints
m.addConstr(quicksum(x[i] for i in range(V)) == B, name="resources")
m.addConstrs((quicksum(coeff2_v[j][i]*x[i] + coeff2_t[j][i]*x[i] for i in range(V)) >= K for j in range(num_samples)), name="HT")
m.addConstrs((quicksum(coeff3_v[j][i]*x[i] + coeff3_t[j][i]*x[i] for i in range(V)) <= L for j in range(num_samples)), name="NHT")
m.addConstrs((error[j] == quicksum(coq_v[j][i]*x[i] + coq_t[j][i]*x[i] for i in range(V)) for j in range(num_samples)), name='error')
m.update()

In [ ]:
m.optimize()

In [ ]:
sol1 = [v.X for k,v in x.items()]
obj_vals1 = [v.X for k,v in error.items()]
k_vals1 = [quicksum(coeff2_v[j][i]*x[i] + coeff2_t[j][i]*x[i] for i in range(V)).getValue() for j in range(num_samples)]
l_vals1 = [quicksum(coeff3_v[j][i]*x[i] + coeff3_t[j][i]*x[i] for i in range(V)).getValue() for j in range(num_samples)]

In [ ]:
with open('lji_vt_frac_v.pkl', 'wb') as f:
    pickle.dump([obj_vals, k_vals, l_vals, sol], f)
# with open('lji-vt_frac_v.pkl', 'rb') as f:
#     obj_vals, k_vals, l_vals, sol = pickle.load(f)

In [ ]:
viridis = cm.get_cmap('viridis', 4)

Create plots for Figure 9

In [ ]:
plt.rcParams['font.size'] = '12'
key1 = 0.7
for key in obj_vals.keys():
    plt.hist((obj_vals[key]-np.mean(obj_vals[key1]))*100/np.mean(obj_vals[key1]), bins=30, alpha=0.75, label=str(key), density=True, color=viridis((key-.6)*1/.3))
# plt.legend(bbox_to_anchor=(1,1), title='Fraction of victims')
plt.xlabel('Errors', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig('lji_errors_vt.pdf', bbox_inches='tight', transparent=True)

In [ ]:
plt.rcParams['font.size'] = '12'
key1 = 0.7
for key in k_vals.keys():
    plt.hist((k_vals[key]-np.mean(k_vals[key1]))*100/np.mean(k_vals[key1]), bins=30, alpha=0.75, label=str(key), density=True, color=viridis((key-.6)*1/.3))
# plt.legend(bbox_to_anchor=(1,1), title='Fraction of victims')
plt.xlabel('Trafficking', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig('lji_K_vt.pdf', bbox_inches='tight', transparent=True)

In [ ]:
plt.rcParams['font.size'] = '12'
key1 = 0.7
for key in l_vals.keys():
    plt.hist((l_vals[key]-np.mean(l_vals[key1]))*100/np.mean(l_vals[key1]), bins=30, alpha=0.75, label=str(key), density=True, color=viridis((key-.6)*1/.3))
plt.legend(bbox_to_anchor=(1,1), title='Fraction of\n victims')
plt.xlabel('Non-trafficking', fontsize=14)
plt.ylabel('Probability', fontsize=14)
plt.savefig('lji_L_vt.pdf', bbox_inches='tight', transparent=True)